In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import matplotlib as plt
import numpy as np
from scipy.interpolate import interp1d
#import cv2
# filename = 'carbide.jpg'
# img=mpimg.imread(filename)
# fig = plt.figure(figsize=(10,5))
# plt.imshow(img)
# plt.plot(30.00649350649357, 624.9545454545455, 'ro')
# plt.plot(32.603896103896176, 117.16233766233768, 'ro')
# plt.plot(1252.0844155844154, 624.9545454545455, 'ro')
# plt.plot(1249.4870129870128, 136.64285714285722, 'ro')
# plt.plot(1249.4870129870128, 117.16, 'bo')
# pts1 = np.float32([[30.00649350649357, 624.9545454545455],\
#                        [32.603896103896176, 117.16233766233768],\
#                        [1252.0844155844154, 624.9545454545455],\
#                        [1249.4870129870128, 136.64285714285722]])
# pts2 = np.float32([[30.00649350649357, 624.9545454545455],\
#                        [30.603896103896176, 117.16233766233768],\
#                        [1252.0844155844154, 624.9545454545455],\
#                        [1249.4870129870128, 117.16233766233768]])

# M = cv2.getPerspectiveTransform(pts1,pts2)

In [3]:
%matplotlib tk
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

def get_pt_onclick(event):
    global ix,iy, rgba
    ix, iy = event.xdata,event.ydata
    rgba = event.inaxes.get_images()[0].get_cursor_data(event).data
    print(ix, iy)
    
def get_pts_manual(event):
    fig.canvas.draw_idle()
    global ix,iy, pts_manual
    if event.key == 'm':
        plt.plot(ix,iy, 'x', color = 'tab:red') 
        pts_manual.append([ix,iy])


def cal_points(event): 
    fig.canvas.draw_idle()
    global ix,iy, calpts
    if len(calpts)<4:
        #print(event.key)
        if event.key == 'enter':
            calpts.append([ix,iy])
            plt.plot(ix,iy, 'rx')
#             if len(calpts)>=4:
#                 plt.close(plt.gcf())


# def bound_points(event): 
#     fig.canvas.draw_idle()
#     global ix,iy, boundpts
#     if len(boundpts)<4:
#         #print(event.key)
#         if event.key == 'b':
#             boundpts.append([ix,iy])
#             plt.plot(ix,iy, 'gx')

                
def line_color(event):
    global ix,iy,rgba,line_col
    if event.key == '-':
        print(event.key)
        line_col = rgba 
        
def mouse_eraser(event):
    fig.canvas.draw_idle()
    global ix,iy, pts
    if event.key == ' ':
        plt.plot(event.xdata,event.ydata, 'bo') 
        try:
            idx = np.where(((pts[0,:] - event.xdata)**2 + (pts[1,:] - event.ydata)**2)<400)[0]
            pts[:, idx] = 0
        except:
            print('bleh')
            
            
def bound_polygon(event):
    fig.canvas.draw_idle()
    global ix,iy, boundpts
    #print(event.key)
    if event.key == 'b':
        boundpts.append([ix,iy])
        plt.plot(ix,iy, 'gx')

In [5]:
# boundry_x = []
# boundry_y = []
# for i in range(len(pts)):
#     print((i+1)%len(pts))
#     f = interp1d([pts[i,1], pts[(i+1)%len(pts),1]], [pts[i,0],pts[(i+1)%len(pts),0]] ,bounds_error=False, fill_value = 'extrapolate')
#     h = np.linspace(pts[i,1],pts[(i+1)%len(pts),1], abs(int(pts[(i+1)%len(pts),1]-pts[i,1])))
#     boundry_x+=list(f(h))
#     boundry_y+=list(h)
    
# plt.figure(2)
# plt.plot(boundry_x, boundry_y)
# plt.plot(pts[:,0], pts[:,1], 'o')

# boundry_x = np.array(boundry_x)
# roi = np.zeros((img.shape[0],img.shape[1]))
# h = np.arange(int(np.min(boundry_y)),int(np.max(boundry_y)))
# for i,y in enumerate(h):
#     xs = np.where((boundry_y-y)**2<1)[0]
#     xmin = int(np.min(boundry_x[xs]))
#     xmax = int(np.max(boundry_x[xs]))
#     roi[y,xmin:xmax] = 1
    
# plt.figure(3)
# plt.imshow(roi)
# plt.colorbar()

In [4]:
def get_path(pts):
    path = 0
    for i in range(len(pts)):
        path+= np.sqrt((pts[i%len(pts),0]-pts[(i+1)%len(pts),0])**2+ (pts[i%len(pts),1]-pts[(i+1)%len(pts),1])**2)
    return path

In [6]:
ls

3ps_49mW_50att_250kHz.npy           IMG_1833.png
4ps_29mW_50att_250kHz.npy           IMG_1835.png
5ps_0.3mW_250kHz.npy                IMG_1836.png
5ps_0.3mW_250kHz_2.npy              IMG_1837.png
5ps_103mW_250kHz.npy                IMG_1838.png
5ps_17mW_50att_250kHz.npy           IMG_1839.png
5ps_18mW_250kHz.npy                 IMG_1840.png
5ps_3.3mW_250kHz.npy                IMG_1841.png
5ps_31mW_250kHz.npy                 IMG_1842.png
5ps_45mW_250kHz.npy                 IMG_1843.png
5ps_58mW_250kHz.npy                 IMG_1844.png
5ps_70mW_250kHz.npy                 IMG_1845.png
5ps_8.5mW_250kHz.npy                IMG_1846.png
5ps_8.5mW_250kHz_2.npy              IMG_1847.png
5ps_84mW_250kHz.npy                 IMG_1848 2.png
5ps_UV_spectra.npy                  IMG_1851.png
6ps_12mW_50att_250kHz.npy           IMG_1852.png
7ps_8.5mW_50att_250kHz.npy          IMG_1853.png
8ps_6mW_50att_250kHz.npy            IMG_1854.png
Bokeh_tools.ipynb                   IMG_1855.png
Digitizer.ipynb   

In [80]:
filename = 'carbide_flat.jpg' #'tangerine_spectrum.png' #'carbide_flat.jpg' #'IMG_1840.png'

In [52]:
#set calpts (enter), boundpts (b), line_col (-)

In [81]:
img=mpimg.imread(filename)
if np.max(img) > 2: img = img/255
fig = plt.figure(figsize=(10,5))
cid = fig.canvas.mpl_connect('button_press_event', get_pt_onclick)
kid = fig.canvas.mpl_connect('key_press_event', cal_points)
#bid = fig.canvas.mpl_connect('key_press_event', bound_points)
bid = fig.canvas.mpl_connect('key_press_event', bound_polygon)
lid = fig.canvas.mpl_connect('key_press_event', line_color)
eid = fig.canvas.mpl_connect('key_press_event', mouse_eraser)
mid = fig.canvas.mpl_connect('key_press_event', get_pts_manual)

ix,iy = 0,0
line_col = [0,0,0,0]
calpts = []
boundpts = []
pts_manual = []
imgplot = plt.imshow(img)
plt.show()

In [82]:
save_name = 'carbide2'

97.01207196877826 627.265656075673
1131.1716644984895 627.265656075673
29.85885167464116 609.3581306639031
29.85885167464116 157.19311401671337
51.12403810111792 614.9542323550812
326.45224130708004 246.73074107556283
596.184342821864 166.14687672259822
762.9481732189712 166.14687672259822
999.1036645866867 535.4895883403524
1226.305393248517 581.3776222080127
1228.5438339249883 625.0272153992017
1213.9939695279252 620.5503340462593
959.9309527484401 619.4311137080236
723.7754613807246 461.6210460168015
480.9046479835954 616.0734526933168
468.59322426300355 620.5503340462593
659.9799021012942 247.84996141379838
-


In [310]:
#roi for 4 boundpts
# mask = np.sum((img - np.tile(line_col,(img.shape[0],img.shape[1],1)))**2, axis = 2)
# mask2 = np.copy(mask)
# mask2[mask2>0.02] = 0
# X,Y=np.meshgrid(np.arange(mask2.shape[1]),np.arange(mask2.shape[0]))
# bb = np.array(boundpts)
# x0 = np.min(bb[:, 0]) #x
# y0 = np.max(bb[:, 1]) #y
# x1 = np.max(bb[:, 0]) #x
# y1 = np.min(bb[:, 1]) #y
# roi = np.zeros(mask2.shape)
# roi[np.where((x0<X) & (y0>Y) & (x1>X) & (y1<Y))[0],np.where((x0<X) & (y0>Y) & (x1>X) & (y1<Y))[1]] = 1
# plt.figure(1)
# pts = np.array([np.where(mask2*roi>0)[1],np.where(mask2*roi>0)[0]])
# plt.plot(pts[0,:], pts[1,:], 'x', color = 'palevioletred')
# plt.show()

In [83]:
#roi for polygon
mask = np.sum((img - np.tile(line_col,(img.shape[0],img.shape[1],1)))**2, axis = 2)
mask2 = np.copy(mask)
mask2[mask2>0.02] = 0
#mask2[mask2>16] = 0
boundpts = np.array(boundpts)
boundry_x = []
boundry_y = []
for i in range(len(boundpts)):
    print((i+1)%len(boundpts))
    f = interp1d([boundpts[i,1], boundpts[(i+1)%len(boundpts),1]], [boundpts[i,0],boundpts[(i+1)%len(boundpts),0]] ,bounds_error=False, fill_value = 'extrapolate')
    h = np.linspace(boundpts[i,1],boundpts[(i+1)%len(boundpts),1], abs(int(boundpts[(i+1)%len(boundpts),1]-boundpts[i,1])))
    boundry_x+=list(f(h))
    boundry_y+=list(h)
    
plt.figure(1)
plt.plot(boundry_x, boundry_y, 'g', alpha = 0.4)

boundry_x = np.array(boundry_x)
roi = np.zeros((img.shape[0],img.shape[1]))
h = np.arange(int(np.floor(np.min(boundry_y))),int(np.ceil(np.max(boundry_y))))
for i,y in enumerate(h):
    xs = np.where((boundry_y-y)**2<1)[0]
    xmin = int(np.min(boundry_x[xs]))
    xmax = int(np.max(boundry_x[xs]))
    roi[y,xmin:xmax] = 1
    
plt.figure(1)
pts = np.array([np.where(mask2*roi>0)[1],np.where(mask2*roi>0)[0]])
plt.plot(pts[0,:], pts[1,:], 'x', color = 'palevioletred', alpha = 0.6)
plt.show()

1
2
3
4
5
6
7
8
9
10
11
0
677.8874275130642 233.30009701673532
677.8874275130642 233.30009701673532
730.4907834101382 205.31958856084486
954.334851057262 593.6890459286044


In [84]:
#manual correction, erase (space), add (m)
if len(pts_manual) > 0: pts = np.hstack((pts,np.array(pts_manual).T))
#pts = np.vstack((pts[0,:][(pts[0,:]>x0)&(pts[1,:]<y0)],pts[1,:][(pts[0,:]>x0)&(pts[1,:]<y0)]))
plt.figure(1)
plt.plot(pts[0,:], pts[1,:], 'gx')
plt.show()

Exception in Tkinter callback
Traceback (most recent call last):
  File "/Users/parisfranz/opt/anaconda3/lib/python3.8/tkinter/__init__.py", line 1883, in __call__
    return self.func(*args)
  File "/Users/parisfranz/opt/anaconda3/lib/python3.8/site-packages/matplotlib/backends/_backend_tk.py", line 293, in button_press_event
    FigureCanvasBase.button_press_event(
  File "/Users/parisfranz/opt/anaconda3/lib/python3.8/site-packages/matplotlib/backend_bases.py", line 1854, in button_press_event
    self.callbacks.process(s, mouseevent)
  File "/Users/parisfranz/opt/anaconda3/lib/python3.8/site-packages/matplotlib/cbook/__init__.py", line 229, in process
    self.exception_handler(exc)
  File "/Users/parisfranz/opt/anaconda3/lib/python3.8/site-packages/matplotlib/cbook/__init__.py", line 81, in _exception_printer
    raise exc
  File "/Users/parisfranz/opt/anaconda3/lib/python3.8/site-packages/matplotlib/cbook/__init__.py", line 224, in process
    func(*args, **kwargs)
  File "<ipytho

In [91]:
#interpolate and save
calval = [1008,1040,0.0,1]
fx = interp1d([calpts[0][0],calpts[1][0]], [calval[0],calval[1]],bounds_error=False, fill_value = 'extrapolate')
fy = interp1d([calpts[2][1],calpts[3][1]],[calval[2],calval[3]],bounds_error=False, fill_value = 'extrapolate')
data = {}
data['x'] = fx(pts[0,:])
data['y'] = fy(pts[1,:])
np.save(save_name+'.npy',data)

In [86]:
plt.figure(3)
plt.plot(data['x'],data['y'], 'o')

In [92]:
data['y'] = data['y'][np.argsort(data['x'])]
data['x'] = data['x'][np.argsort(data['x'])]
plt.figure(4)
plt.plot(data['x'],data['y'], '-')
plt.plot(d2['x'], d2['y'])
plt.grid()


In [93]:
np.save(save_name+'.npy',data)

In [39]:
np.max(data['y'])

0.9970269167278062

In [98]:
d = np.load('tangerine_spec.npy', allow_pickle = True)[()]
d2 = np.load('carbide2.npy', allow_pickle = True)[()]
plt.plot(d2['x'], d2['y']/np.max(d2['y']), label = 'carbide')
plt.plot(d['x'], d['y']/np.max(d['y']), label = 'tangerine')
plt.grid()
plt.legend()